In [29]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
# Config
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [201]:
# Utilities
def merge(pr, haiti, rd):
    df = pd.merge(pr_pib, haiti_pib, on='Fecha')
    df = pd.merge(df, rd_pib, on='Fecha')
    
    df = df.rename(columns={
        'Cambio_x': 'Puerto Rico',
        'Cambio_y': 'Haiti',
        'Cambio': 'Republica Dominicana',
    })
    
    df = df[['Fecha', 'Puerto Rico', 'Haiti', 'Republica Dominicana']]
    
    df = df[(df['Fecha'] >= 2000)].reset_index(drop=True)
    
    return df

In [152]:
def load_pib(gdp_type='constant'):

    df = pd.read_excel(f'./input/pib_{gdp_type}.xls', skiprows=3)
    df = df.drop(df.columns[1:4], axis=1)
    df = df.T
    df.columns = df.iloc[0]
    df = df[1:]

    haiti = df['Haiti'].dropna()
    pr = df['Puerto Rico'].dropna()
    rd = df['Dominican Republic'].dropna()

    haiti = haiti.reset_index()
    haiti['Cambio'] = haiti['Haiti'].pct_change()
    haiti = haiti.rename(columns={'Haiti':'PIB', 'index':'Fecha'})
    haiti['Fecha'] = haiti.Fecha.astype(int)

    pr = pr.reset_index()
    pr['Cambio'] = pr['Puerto Rico'].pct_change()
    pr = pr.rename(columns={'Puerto Rico':'PIB', 'index':'Fecha'})
    pr['Fecha'] = pr.Fecha.astype(int)

    rd = rd.reset_index()
    rd['Cambio'] = rd['Dominican Republic'].pct_change()
    rd = rd.rename(columns={'Dominican Republic':'PIB', 'index':'Fecha'})
    rd['Fecha'] = rd.Fecha.astype(int)

    return haiti, pr, rd
    
haiti_pib, pr_pib, rd_pib = load_pib()

In [202]:
merge(pr_pib, haiti_pib, rd_pib)

,Fecha,Puerto Rico,Haiti,Republica Dominicana
0,2000,0.032720,0.008701,0.046618
1,2001,0.063135,-0.003427,0.024595
2,2002,0.009165,0.010518,0.044951
3,2003,0.000534,0.034808,-0.013457
4,2004,0.087487,-0.013179,0.025701
5,2005,-0.019869,0.030722,0.094282
6,2006,-0.014094,0.017688,0.091744
7,2007,-0.011628,0.047065,0.074160
8,2008,-0.018441,0.026580,0.032095
9,2009,-0.019524,0.058904,0.009462


In [177]:
def plot_pib(df, country, ano):
    # Get only past 2000
    if country == 'Republica Dominicana':
        df = df[df['Fecha'] >= 2000]
    else:
        df = df[(df['Fecha'] >= 2000) & (df['Fecha'] < 2020)]

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=df.Fecha, y=df.PIB, name="PIB", line_color="green"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=df.Fecha, y=df.Cambio, name="Cambio del PIB", line_dash="dash", line_color="green"),
        secondary_y=True,
    )

    fig.add_vline(x=ano, line_width=3, line_color="darkred")

    # Add figure title
    fig.update_layout(
        title_text=f"PIB y su cambio en {country}",
        width=800
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Fecha")#, ticklabelstep=5)

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>PIB</b> USD$", secondary_y=False)
    fig.update_yaxes(title_text="<b>Cambio</b> del PIB %", secondary_y=True)

    fig.show()

plot_pib(pr_pib, 'Puerto Rico', 2017)
plot_pib(haiti_pib, 'Haiti', 2010)
plot_pib(rd_pib, 'Republica Dominicana', 2010)

In [154]:
def load_unemployment():
    df = pd.read_excel('./input/unemployment.xls', skiprows=3)
    df = df.drop(df.columns[1:4], axis=1)
    df = df.T
    df.columns = df.iloc[0]
    df = df[1:]

    haiti = df['Haiti'].dropna()
    pr = df['Puerto Rico'].dropna()
    rd = df['Dominican Republic'].dropna()

    haiti = haiti.reset_index()
    haiti['Cambio'] = haiti['Haiti'].diff()
    haiti = haiti.rename(columns={'Haiti':'Desempleo', 'index':'Fecha'})
    haiti['Fecha'] = haiti.Fecha.astype(int)

    pr = pr.reset_index()
    pr['Cambio'] = pr['Puerto Rico'].diff()
    pr = pr.rename(columns={'Puerto Rico':'Desempleo', 'index':'Fecha'})
    pr['Fecha'] = pr.Fecha.astype(int)

    rd = rd.reset_index()
    rd['Cambio'] = rd['Dominican Republic'].diff()
    rd = rd.rename(columns={'Dominican Republic':'Desempleo', 'index':'Fecha'})
    rd['Fecha'] = rd.Fecha.astype(int)

    return haiti, pr, rd
    
haiti_unem, pr_unem, rd_unem = load_unemployment()

In [203]:
merge(pr_unem, haiti_unem, rd_unem)

,Fecha,Puerto Rico,Haiti,Republica Dominicana
0,2000,0.032720,0.008701,0.046618
1,2001,0.063135,-0.003427,0.024595
2,2002,0.009165,0.010518,0.044951
3,2003,0.000534,0.034808,-0.013457
4,2004,0.087487,-0.013179,0.025701
5,2005,-0.019869,0.030722,0.094282
6,2006,-0.014094,0.017688,0.091744
7,2007,-0.011628,0.047065,0.074160
8,2008,-0.018441,0.026580,0.032095
9,2009,-0.019524,0.058904,0.009462


In [178]:
def plot_unem(df, country, ano):
    # Get only past 2000
    if country == 'Republica Dominicana':
        df = df[df['Fecha'] >= 2000]
    else:
        df = df[(df['Fecha'] >= 2000) & (df['Fecha'] < 2020)]

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=df.Fecha, y=df.Desempleo, name="Tasa de Desempleo", line_color="darkblue"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=df.Fecha, y=df.Cambio, name="Cambio del Desempleo", line_dash="dash", line_color="darkblue"),
        secondary_y=True,
    )

    fig.add_vline(x=ano, line_width=3, line_color="darkred")

    # Add figure title
    fig.update_layout(
        title_text=f"Desempleo y su cambio en {country}",
        width=800
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Fecha")#, ticklabelstep=5)

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Desempleo</b> %", secondary_y=False)
    fig.update_yaxes(title_text="<b>Cambio</b> del Desempleo %", secondary_y=True)

    fig.show()

plot_unem(pr_unem, 'Puerto Rico', 2017)
plot_unem(haiti_unem, 'Haiti', 2010)
plot_unem(rd_unem, 'Republica Dominicana', 2010)

In [156]:
def load_inflation():
    df = pd.read_excel('./input/inflation.xls', skiprows=3)
    df = df.drop(df.columns[1:4], axis=1)
    df = df.T
    df.columns = df.iloc[0]
    df = df[1:]

    haiti = df['Haiti'].dropna()
    pr = df['United States'].dropna()
    rd = df['Dominican Republic'].dropna()

    haiti = haiti.reset_index()
    haiti['Cambio'] = haiti['Haiti'].diff()
    haiti = haiti.rename(columns={'Haiti':'Inflacion', 'index':'Fecha'})
    haiti['Fecha'] = haiti.Fecha.astype(int)

    pr = pr.reset_index()
    pr['Cambio'] = pr['United States'].diff()
    pr = pr.rename(columns={'United States':'Inflacion', 'index':'Fecha'})
    pr['Fecha'] = pr.Fecha.astype(int)

    rd = rd.reset_index()
    rd['Cambio'] = rd['Dominican Republic'].diff()
    rd = rd.rename(columns={'Dominican Republic':'Inflacion', 'index':'Fecha'})
    rd['Fecha'] = rd.Fecha.astype(int)

    return haiti, pr, rd
    
haiti_inf, pr_inf, rd_inf = load_inflation()

In [204]:
merge(pr_inf, haiti_inf, rd_inf)

,Fecha,Puerto Rico,Haiti,Republica Dominicana
0,2000,0.032720,0.008701,0.046618
1,2001,0.063135,-0.003427,0.024595
2,2002,0.009165,0.010518,0.044951
3,2003,0.000534,0.034808,-0.013457
4,2004,0.087487,-0.013179,0.025701
5,2005,-0.019869,0.030722,0.094282
6,2006,-0.014094,0.017688,0.091744
7,2007,-0.011628,0.047065,0.074160
8,2008,-0.018441,0.026580,0.032095
9,2009,-0.019524,0.058904,0.009462


In [179]:
def plot_inf(df, country, ano):
    # Get only past 2000
    if country == 'Republica Dominicana':
        df = df[df['Fecha'] >= 2000]
    else:
        df = df[(df['Fecha'] >= 2000) & (df['Fecha'] < 2020)]

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=df.Fecha, y=df.Inflacion, name="Inflacion", line_color="sandybrown"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=df.Fecha, y=df.Cambio, name="Cambio del Desempleo", line_dash="dash", line_color="sandybrown"),
        secondary_y=True,
    )

    fig.add_vline(x=ano, line_width=3, line_color="darkred")

    # Add figure title
    fig.update_layout(
        title_text=f"Inflacion y su cambio en {country}",
        width=800
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Fecha")#, ticklabelstep=5)

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Inflacion</b> %", secondary_y=False)
    fig.update_yaxes(title_text="<b>Cambio</b> de la Inflacion %", secondary_y=True)

    fig.show()

plot_inf(pr_inf, 'Estados Unidos', 2017)
plot_inf(haiti_inf, 'Haiti', 2010)
plot_inf(rd_inf, 'Republica Dominicana', 2017)

In [158]:
def load_services():
    df = pd.read_excel('./input/services as percent.xls', skiprows=3)
    df = df.drop(df.columns[1:4], axis=1)
    df = df.T
    df.columns = df.iloc[0]
    df = df[1:]

    haiti = df['Haiti'].dropna()
    pr = df['Puerto Rico'].dropna()
    rd = df['Dominican Republic'].dropna()

    haiti = haiti.reset_index()
    haiti = haiti.rename(columns={'Haiti':'SERV%', 'index':'Fecha'})
    haiti['Fecha'] = haiti.Fecha.astype(int)
    haiti = pd.merge(haiti, haiti_pib, on='Fecha')
    haiti['Servicios'] = haiti['SERV%'] * haiti['PIB'] / 100
    haiti = haiti[['Fecha','Servicios']]
    haiti['Cambio'] = haiti['Servicios'].pct_change()

    pr = pr.reset_index()
    pr = pr.rename(columns={'Puerto Rico':'SERV%', 'index':'Fecha'})
    pr['Fecha'] = pr.Fecha.astype(int)
    pr = pd.merge(pr, pr_pib, on='Fecha')
    pr['Servicios'] = pr['SERV%'] * pr['PIB'] / 100
    pr = pr[['Fecha','Servicios']]
    pr['Cambio'] = pr['Servicios'].pct_change()

    rd = rd.reset_index()
    rd = rd.rename(columns={'Dominican Republic':'SERV%', 'index':'Fecha'})
    rd['Fecha'] = rd.Fecha.astype(int)
    rd = pd.merge(rd, rd_pib, on='Fecha')
    rd['Servicios'] = rd['SERV%'] * rd['PIB'] / 100
    rd = rd[['Fecha','Servicios']]
    rd['Cambio'] = rd['Servicios'].pct_change()

    return haiti, pr, rd
    
haiti_ser, pr_ser, rd_ser = load_services()

In [206]:
merge(pr_ser, haiti_ser, rd_ser)

,Fecha,Puerto Rico,Haiti,Republica Dominicana
0,2000,0.032720,0.008701,0.046618
1,2001,0.063135,-0.003427,0.024595
2,2002,0.009165,0.010518,0.044951
3,2003,0.000534,0.034808,-0.013457
4,2004,0.087487,-0.013179,0.025701
5,2005,-0.019869,0.030722,0.094282
6,2006,-0.014094,0.017688,0.091744
7,2007,-0.011628,0.047065,0.074160
8,2008,-0.018441,0.026580,0.032095
9,2009,-0.019524,0.058904,0.009462


In [180]:
def plot_serv(df, country, ano):
    # Get only past 2000
    if country == 'Republica Dominicana':
        df = df[df['Fecha'] >= 2000]
    else:
        df = df[(df['Fecha'] >= 2000) & (df['Fecha'] < 2020)]

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=df.Fecha, y=df.Servicios, name="Servicios", line_color="indianred"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=df.Fecha, y=df.Cambio, name="Cambio de Servicios", line_dash="dash", line_color="indianred"),
        secondary_y=True,
    )

    fig.add_vline(x=ano, line_width=3, line_color="darkred")

    # Add figure title
    fig.update_layout(
        title_text=f"Servicios y su cambio en {country}",
        width=800
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Fecha")#, ticklabelstep=5)

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Servicios</b> USD$", secondary_y=False)
    fig.update_yaxes(title_text="<b>Cambio</b> de Servicios %", secondary_y=True)

    fig.show()

plot_serv(pr_ser, 'Puerto Rico', 2017)
plot_serv(haiti_ser, 'Haiti', 2010)
plot_serv(rd_ser, 'Republica Dominicana', 2010)

In [161]:
def load_manufacturing():
    df = pd.read_excel('./input/manufacturing.xls', skiprows=3)
    df = df.drop(df.columns[1:4], axis=1)
    df = df.T
    df.columns = df.iloc[0]
    df = df[1:]

    haiti = df['Haiti'].dropna()
    pr = df['Puerto Rico'].dropna()
    rd = df['Dominican Republic'].dropna()

    haiti = haiti.reset_index()
    haiti = haiti.rename(columns={'Haiti':'Man', 'index':'Fecha'})
    haiti['Fecha'] = haiti.Fecha.astype(int)
    haiti = pd.merge(haiti, haiti_pib, on='Fecha')
    haiti['Manufactura'] = haiti['Man'] * haiti['PIB'] / 100
    haiti = haiti[['Fecha','Manufactura']]
    haiti['Cambio'] = haiti['Manufactura'].pct_change()

    pr = pr.reset_index()
    pr = pr.rename(columns={'Puerto Rico':'Man', 'index':'Fecha'})
    pr['Fecha'] = pr.Fecha.astype(int)
    pr = pd.merge(pr, pr_pib, on='Fecha')
    pr['Manufactura'] = pr['Man'] * pr['PIB'] / 100
    pr = pr[['Fecha','Manufactura']]
    pr['Cambio'] = pr['Manufactura'].pct_change()

    rd = rd.reset_index()
    rd = rd.rename(columns={'Dominican Republic':'Man', 'index':'Fecha'})
    rd['Fecha'] = rd.Fecha.astype(int)
    rd = pd.merge(rd, rd_pib, on='Fecha')
    rd['Manufactura'] = rd['Man'] * rd['PIB'] / 100
    rd = rd[['Fecha','Manufactura']]
    rd['Cambio'] = rd['Manufactura'].pct_change()

    return haiti, pr, rd
    
haiti_man, pr_man, rd_man = load_manufacturing()

In [207]:
merge(pr_man, haiti_man, rd_man)

,Fecha,Puerto Rico,Haiti,Republica Dominicana
0,2000,0.032720,0.008701,0.046618
1,2001,0.063135,-0.003427,0.024595
2,2002,0.009165,0.010518,0.044951
3,2003,0.000534,0.034808,-0.013457
4,2004,0.087487,-0.013179,0.025701
5,2005,-0.019869,0.030722,0.094282
6,2006,-0.014094,0.017688,0.091744
7,2007,-0.011628,0.047065,0.074160
8,2008,-0.018441,0.026580,0.032095
9,2009,-0.019524,0.058904,0.009462


In [181]:
def plot_man(df, country, ano):
    # Get only past 2000
    if country == 'Republica Dominicana':
        df = df[df['Fecha'] >= 2000]
    else:
        df = df[(df['Fecha'] >= 2000) & (df['Fecha'] < 2020)]

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=df.Fecha, y=df.Manufactura, name="Manufactura", line_color="black"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=df.Fecha, y=df.Cambio, name="Cambio de Manufactura", line_dash="dash", line_color="black"),
        secondary_y=True,
    )

    fig.add_vline(x=ano, line_width=3, line_color="darkred")

    # Add figure title
    fig.update_layout(
        title_text=f"Manufactura y su cambio en {country}",
        width=800
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Fecha")#, ticklabelstep=5)

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Manufactura</b> USD$", secondary_y=False)
    fig.update_yaxes(title_text="<b>Cambio</b> de Manufactura %", secondary_y=True)

    fig.show()

plot_man(pr_man, 'Puerto Rico', 2017)
plot_man(haiti_man, 'Haiti', 2010)
plot_man(rd_man, 'Republica Dominicana', 2010)

In [164]:
def load_industria():
    df = pd.read_excel('./input/industria.xls', skiprows=3)
    df = df.drop(df.columns[1:4], axis=1)
    df = df.T
    df.columns = df.iloc[0]
    df = df[1:]

    haiti = df['Haiti'].dropna()
    pr = df['Puerto Rico'].dropna()
    rd = df['Dominican Republic'].dropna()

    haiti = haiti.reset_index()
    haiti = haiti.rename(columns={'Haiti':'Man', 'index':'Fecha'})
    haiti['Fecha'] = haiti.Fecha.astype(int)
    haiti = pd.merge(haiti, haiti_pib, on='Fecha')
    haiti['Industria'] = haiti['Man'] * haiti['PIB'] / 100
    haiti = haiti[['Fecha','Industria']]
    haiti['Cambio'] = haiti['Industria'].pct_change()

    pr = pr.reset_index()
    pr = pr.rename(columns={'Puerto Rico':'Man', 'index':'Fecha'})
    pr['Fecha'] = pr.Fecha.astype(int)
    pr = pd.merge(pr, pr_pib, on='Fecha')
    pr['Industria'] = pr['Man'] * pr['PIB'] / 100
    pr = pr[['Fecha','Industria']]
    pr['Cambio'] = pr['Industria'].pct_change()

    rd = rd.reset_index()
    rd = rd.rename(columns={'Dominican Republic':'Man', 'index':'Fecha'})
    rd['Fecha'] = rd.Fecha.astype(int)
    rd = pd.merge(rd, rd_pib, on='Fecha')
    rd['Industria'] = rd['Man'] * rd['PIB'] / 100
    rd = rd[['Fecha','Industria']]
    rd['Cambio'] = rd['Industria'].pct_change()

    return haiti, pr, rd
    
haiti_ind, pr_ind, rd_ind = load_industria()

In [208]:
merge(pr_ind, haiti_ind, rd_ind)

,Fecha,Puerto Rico,Haiti,Republica Dominicana
0,2000,0.032720,0.008701,0.046618
1,2001,0.063135,-0.003427,0.024595
2,2002,0.009165,0.010518,0.044951
3,2003,0.000534,0.034808,-0.013457
4,2004,0.087487,-0.013179,0.025701
5,2005,-0.019869,0.030722,0.094282
6,2006,-0.014094,0.017688,0.091744
7,2007,-0.011628,0.047065,0.074160
8,2008,-0.018441,0.026580,0.032095
9,2009,-0.019524,0.058904,0.009462


In [182]:
def plot_ind(df, country, ano):
    # Get only past 2000
    if country == 'Republica Dominicana':
        df = df[df['Fecha'] >= 2000]
    else:
        df = df[(df['Fecha'] >= 2000) & (df['Fecha'] < 2020)]

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=df.Fecha, y=df.Industria, name="Industria", line_color="grey"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=df.Fecha, y=df.Cambio, name="Cambio de Industria", line_dash="dash", line_color="grey"),
        secondary_y=True,
    )

    fig.add_vline(x=ano, line_width=3, line_color="darkred")

    # Add figure title
    fig.update_layout(
        title_text=f"Industria y su cambio en {country}",
        width=800
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Fecha")#, ticklabelstep=5)

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Industria</b> USD$", secondary_y=False)
    fig.update_yaxes(title_text="<b>Cambio</b> de Industria %", secondary_y=True)

    fig.show()

plot_ind(pr_ind, 'Puerto Rico', 2017)
plot_ind(haiti_ind, 'Haiti', 2010)
plot_ind(rd_ind, 'Republica Dominicana', 2017)

In [166]:
def load_agricultura():
    df = pd.read_excel('./input/agricultura.xls', skiprows=3)
    df = df.drop(df.columns[1:4], axis=1)
    df = df.T
    df.columns = df.iloc[0]
    df = df[1:]

    haiti = df['Haiti'].dropna()
    pr = df['Puerto Rico'].dropna()
    rd = df['Dominican Republic'].dropna()

    haiti = haiti.reset_index()
    haiti = haiti.rename(columns={'Haiti':'Man', 'index':'Fecha'})
    haiti['Fecha'] = haiti.Fecha.astype(int)
    haiti = pd.merge(haiti, haiti_pib, on='Fecha')
    haiti['Agricultura'] = haiti['Man'] * haiti['PIB'] / 100
    haiti = haiti[['Fecha','Agricultura']]
    haiti['Cambio'] = haiti['Agricultura'].pct_change()

    pr = pr.reset_index()
    pr = pr.rename(columns={'Puerto Rico':'Man', 'index':'Fecha'})
    pr['Fecha'] = pr.Fecha.astype(int)
    pr = pd.merge(pr, pr_pib, on='Fecha')
    pr['Agricultura'] = pr['Man'] * pr['PIB'] / 100
    pr = pr[['Fecha','Agricultura']]
    pr['Cambio'] = pr['Agricultura'].pct_change()

    rd = rd.reset_index()
    rd = rd.rename(columns={'Dominican Republic':'Man', 'index':'Fecha'})
    rd['Fecha'] = rd.Fecha.astype(int)
    rd = pd.merge(rd, rd_pib, on='Fecha')
    rd['Agricultura'] = rd['Man'] * rd['PIB'] / 100
    rd = rd[['Fecha','Agricultura']]
    rd['Cambio'] = rd['Agricultura'].pct_change()

    return haiti, pr, rd
    
haiti_agr, pr_agr, rd_agr = load_agricultura()

In [209]:
merge(pr_agr, haiti_agr, rd_agr)

,Fecha,Puerto Rico,Haiti,Republica Dominicana
0,2000,0.032720,0.008701,0.046618
1,2001,0.063135,-0.003427,0.024595
2,2002,0.009165,0.010518,0.044951
3,2003,0.000534,0.034808,-0.013457
4,2004,0.087487,-0.013179,0.025701
5,2005,-0.019869,0.030722,0.094282
6,2006,-0.014094,0.017688,0.091744
7,2007,-0.011628,0.047065,0.074160
8,2008,-0.018441,0.026580,0.032095
9,2009,-0.019524,0.058904,0.009462


In [183]:
def plot_agr(df, country, ano):
    # Get only past 2000
    if country == 'Republica Dominicana':
        df = df[df['Fecha'] >= 2000]
    else:
        df = df[(df['Fecha'] >= 2000) & (df['Fecha'] < 2020)]

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=df.Fecha, y=df.Agricultura, name="Agricultura", line_color="yellowgreen"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=df.Fecha, y=df.Cambio, name="Cambio de Agricultura", line_dash="dash", line_color="yellowgreen"),
        secondary_y=True,
    )

    fig.add_vline(x=ano, line_width=3, line_color="darkred")

    # Add figure title
    fig.update_layout(
        title_text=f"Agricultura y su cambio en {country}",
        width=800
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Fecha")#, ticklabelstep=5)

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Agricultura</b> USD$", secondary_y=False)
    fig.update_yaxes(title_text="<b>Cambio</b> de Agricultura %", secondary_y=True)

    fig.show()

plot_agr(pr_agr, 'Puerto Rico', 2017)
plot_agr(haiti_agr, 'Haiti', 2010)
plot_agr(rd_agr, 'Republica Dominicana', 2010)

In [168]:
def merge_data():
    haiti = pd.merge(haiti_agr, haiti_ind, on='Fecha')
    haiti = pd.merge(haiti, haiti_ser, on='Fecha')
    haiti = haiti[['Fecha', 'Agricultura', 'Industria', 'Servicios']]
    
    pr = pd.merge(pr_agr, pr_ind, on='Fecha')
    pr = pd.merge(pr, pr_ser, on='Fecha')
    pr = pr[['Fecha', 'Agricultura', 'Industria', 'Servicios']]

    rd = pd.merge(rd_agr, rd_ind, on='Fecha')
    rd = pd.merge(rd, rd_ser, on='Fecha')
    rd = rd[['Fecha', 'Agricultura', 'Industria', 'Servicios']]
    
    return pr, haiti, rd

pr_merged, haiti_merged, rd_merged = merge_data()

In [184]:
def plot_merged(df, country, ano):
    # Get only past 2000
    if country == 'Republica Dominicana':
        df = df[df['Fecha'] >= 2000]
    else:
        df = df[(df['Fecha'] >= 2000) & (df['Fecha'] < 2020)]

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Bar(x=df.Fecha, y=df.Agricultura, name="Agricultura", marker_color='yellowgreen'),
        secondary_y=False,
    )

    fig.add_trace(
        go.Bar(x=df.Fecha, y=df.Industria, name="Industria", marker_color='grey'),
        secondary_y=False,
    )

    fig.add_trace(
        go.Bar(x=df.Fecha, y=df.Servicios, name="Servicios", marker_color='indianred'),
        secondary_y=False,
    )

    fig.add_vline(x=ano, line_width=3, line_color="darkred")

    # Add figure title
    fig.update_layout(
        title_text=f"PIB desagregado en {country}",
        width=800,
        barmode='stack'
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Fecha")#, ticklabelstep=5)

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>PIB</b> USD$", secondary_y=False)
    fig.update_yaxes(title_text="<b>Cambio</b> de PIB %", secondary_y=True)

    fig.show()

plot_merged(pr_merged, 'Puerto Rico', 2017)
plot_merged(haiti_merged, 'Haiti', 2010)
plot_merged(rd_merged, 'Republica Dominicana', 2008)

In [171]:
def load_consumo():
    df = pd.read_excel('./input/consumo.xls', skiprows=3)
    df = df.drop(df.columns[1:4], axis=1)
    df = df.T
    df.columns = df.iloc[0]
    df = df[1:]

    haiti = df['Haiti'].dropna()
    pr = df['Puerto Rico'].dropna()
    rd = df['Dominican Republic'].dropna()

    haiti = haiti.reset_index()
    haiti = haiti.rename(columns={'Haiti':'Man', 'index':'Fecha'})
    haiti['Fecha'] = haiti.Fecha.astype(int)
    haiti = pd.merge(haiti, haiti_pib, on='Fecha')
    haiti['Consumo'] = haiti['Man'] * haiti['PIB'] / 100
    haiti = haiti[['Fecha','Consumo']]
    haiti['Cambio'] = haiti['Consumo'].pct_change()

    pr = pr.reset_index()
    pr = pr.rename(columns={'Puerto Rico':'Man', 'index':'Fecha'})
    pr['Fecha'] = pr.Fecha.astype(int)
    pr = pd.merge(pr, pr_pib, on='Fecha')
    pr['Consumo'] = pr['Man'] * pr['PIB'] / 100
    pr = pr[['Fecha','Consumo']]
    pr['Cambio'] = pr['Consumo'].pct_change()

    rd = rd.reset_index()
    rd = rd.rename(columns={'Dominican Republic':'Man', 'index':'Fecha'})
    rd['Fecha'] = rd.Fecha.astype(int)
    rd = pd.merge(rd, rd_pib, on='Fecha')
    rd['Consumo'] = rd['Man'] * rd['PIB'] / 100
    rd = rd[['Fecha','Consumo']]
    rd['Cambio'] = rd['Consumo'].pct_change()

    return haiti, pr, rd
    
haiti_cons, pr_cons, rd_cons = load_consumo()

In [210]:
merge(pr_cons, haiti_cons, rd_cons)

,Fecha,Puerto Rico,Haiti,Republica Dominicana
0,2000,0.032720,0.008701,0.046618
1,2001,0.063135,-0.003427,0.024595
2,2002,0.009165,0.010518,0.044951
3,2003,0.000534,0.034808,-0.013457
4,2004,0.087487,-0.013179,0.025701
5,2005,-0.019869,0.030722,0.094282
6,2006,-0.014094,0.017688,0.091744
7,2007,-0.011628,0.047065,0.074160
8,2008,-0.018441,0.026580,0.032095
9,2009,-0.019524,0.058904,0.009462


In [185]:
def plot_cons(df, country, ano):
    # Get only past 2000
    if country == 'Republica Dominicana':
        df = df[df['Fecha'] >= 2000]
    else:
        df = df[(df['Fecha'] >= 2000) & (df['Fecha'] < 2020)]

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=df.Fecha, y=df.Consumo, name="Consumo", line_color="slateblue"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=df.Fecha, y=df.Cambio, name="Cambio de Consumo", line_dash="dash", line_color="slateblue"),
        secondary_y=True,
    )

    fig.add_vline(x=ano, line_width=3, line_color="darkred")

    # Add figure title
    fig.update_layout(
        title_text=f"Consumo y su cambio en {country}",
        width=800
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Fecha")#, ticklabelstep=5)

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Consumo</b> USD$", secondary_y=False)
    fig.update_yaxes(title_text="<b>Cambio</b> de Consumo %", secondary_y=True)

    fig.show()

plot_cons(pr_cons, 'Puerto Rico', 2017)
plot_cons(haiti_cons, 'Haiti', 2010)
plot_cons(rd_cons, 'Republica Dominicana', 2017)

In [173]:
def load_exports():
    df = pd.read_excel('./input/exports haiti.xls', skiprows=3)
    df = df.drop(df.columns[1:4], axis=1)
    df = df.T
    df.columns = df.iloc[0]
    df = df[1:]

    haiti = df['Haiti'].dropna()
    rd = df['Dominican Republic'].dropna()

    haiti = haiti.reset_index()
    haiti = haiti.rename(columns={'Haiti':'Man', 'index':'Fecha'})
    haiti['Fecha'] = haiti.Fecha.astype(int)
    haiti = pd.merge(haiti, haiti_pib, on='Fecha')
    haiti['Exportacion'] = haiti['Man'] * haiti['PIB'] / 100
    haiti = haiti[['Fecha','Exportacion']]
    haiti['Cambio'] = haiti['Exportacion'].pct_change()

    rd = rd.reset_index()
    rd = rd.rename(columns={'Dominican Republic':'Man', 'index':'Fecha'})
    rd['Fecha'] = rd.Fecha.astype(int)
    rd = pd.merge(rd, rd_pib, on='Fecha')
    rd['Exportacion'] = rd['Man'] * rd['PIB'] / 100
    rd = rd[['Fecha','Exportacion']]
    rd['Cambio'] = rd['Exportacion'].pct_change()

    pr = pd.read_excel('./input/exports pr.xlsx')

    return haiti, pr, rd
    
haiti_exp, pr_exp, rd_exp = load_exports()

In [211]:
merge(pr_exp, haiti_exp, rd_exp)

,Fecha,Puerto Rico,Haiti,Republica Dominicana
0,2000,0.032720,0.008701,0.046618
1,2001,0.063135,-0.003427,0.024595
2,2002,0.009165,0.010518,0.044951
3,2003,0.000534,0.034808,-0.013457
4,2004,0.087487,-0.013179,0.025701
5,2005,-0.019869,0.030722,0.094282
6,2006,-0.014094,0.017688,0.091744
7,2007,-0.011628,0.047065,0.074160
8,2008,-0.018441,0.026580,0.032095
9,2009,-0.019524,0.058904,0.009462


In [186]:
def plot_exp(df, country, ano):
    # Get only past 2000
    if country == 'Republica Dominicana':
        df = df[df['Fecha'] >= 2000]
    else:
        df = df[(df['Fecha'] >= 2000) & (df['Fecha'] < 2020)]

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=df.Fecha, y=df.Exportacion, name="Exportacion", line_color="#32CD32"), #800000
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=df.Fecha, y=df.Cambio, name="Cambio de Exportacion", line_dash="dash", line_color="#32CD32"),
        secondary_y=True,
    )

    fig.add_vline(x=ano, line_width=3, line_color="darkred")

    # Add figure title
    fig.update_layout(
        title_text=f"Exportacion y su cambio en {country}",
        width=800
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Fecha")#, ticklabelstep=5)

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Exportacion</b> USD$", secondary_y=False)
    fig.update_yaxes(title_text="<b>Cambio</b> de Exportacion %", secondary_y=True)

    fig.show()

plot_exp(pr_exp, 'Puerto Rico', 2017)
plot_exp(haiti_exp, 'Haiti', 2010)
plot_exp(rd_exp, 'Republica Dominicana', 2010)

In [175]:
def load_imports():
    df = pd.read_excel('./input/imports haiti.xls', skiprows=3)
    df = df.drop(df.columns[1:4], axis=1)
    df = df.T
    df.columns = df.iloc[0]
    df = df[1:]

    haiti = df['Haiti'].dropna()
    rd = df['Dominican Republic'].dropna()

    haiti = haiti.reset_index()
    haiti = haiti.rename(columns={'Haiti':'Man', 'index':'Fecha'})
    haiti['Fecha'] = haiti.Fecha.astype(int)
    haiti = pd.merge(haiti, haiti_pib, on='Fecha')
    haiti['Importacion'] = haiti['Man'] * haiti['PIB'] / 100
    haiti = haiti[['Fecha','Importacion']]
    haiti['Cambio'] = haiti['Importacion'].pct_change()

    rd = rd.reset_index()
    rd = rd.rename(columns={'Dominican Republic':'Man', 'index':'Fecha'})
    rd['Fecha'] = rd.Fecha.astype(int)
    rd = pd.merge(rd, rd_pib, on='Fecha')
    rd['Importacion'] = rd['Man'] * rd['PIB'] / 100
    rd = rd[['Fecha','Importacion']]
    rd['Cambio'] = rd['Importacion'].pct_change()

    pr = pd.read_excel('./input/imports pr.xlsx')

    return haiti, pr, rd
    
haiti_imp, pr_imp, rd_imp = load_imports()

In [212]:
merge(pr_imp, haiti_imp, rd_imp)

,Fecha,Puerto Rico,Haiti,Republica Dominicana
0,2000,0.032720,0.008701,0.046618
1,2001,0.063135,-0.003427,0.024595
2,2002,0.009165,0.010518,0.044951
3,2003,0.000534,0.034808,-0.013457
4,2004,0.087487,-0.013179,0.025701
5,2005,-0.019869,0.030722,0.094282
6,2006,-0.014094,0.017688,0.091744
7,2007,-0.011628,0.047065,0.074160
8,2008,-0.018441,0.026580,0.032095
9,2009,-0.019524,0.058904,0.009462


In [187]:
def plot_imp(df, country, ano):
    # Get only past 2000
    if country == 'Republica Dominicana':
        df = df[df['Fecha'] >= 2000]
    else:
        df = df[(df['Fecha'] >= 2000) & (df['Fecha'] < 2020)]

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=df.Fecha, y=df.Importacion, name="Importacion", line_color="#888000"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=df.Fecha, y=df.Cambio, name="Cambio de Importacion", line_dash="dash", line_color="#888000"),
        secondary_y=True,
    )

    fig.add_vline(x=ano, line_width=3, line_color="darkred")

    # Add figure title
    fig.update_layout(
        title_text=f"Importacion y su cambio en {country}",
        width=800
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Fecha")#, ticklabelstep=5)

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>Importacion</b> USD$", secondary_y=False)
    fig.update_yaxes(title_text="<b>Cambio</b> de Importacion %", secondary_y=True)

    fig.show()

plot_imp(pr_imp, 'Puerto Rico', 2017)
plot_imp(haiti_imp, 'Haiti', 2010)
plot_imp(rd_imp, 'Republica Dominicana', 2010)